# Исследование препроцессинга на распознавание типа и номера документов прототипа OCR. Детекция OpenVINO. Распознавание PaddleOCR 
- Автор: Кирилл Киселев
- Дата начала: 04.05.2022
- Описание исследования: в датасете содержится 203 изображения, 10 из которых отфильтруются по детектору блюра. Остальные изображения хорошо читаемые. Необходимо довести точность их распознавания до 100%.

## Загрузка библиотек

In [1]:
import os
import pandas as pd
import cv2
import numpy as np

## Глобальные переменные

In [2]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [3]:
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_1.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_2.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_3.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_4.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_5.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_6.txt'
TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_7.txt'
SAVE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results'
DATA_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

## Загрузка данных и подготовка датасета

In [4]:
filename_list = []
results_list = []

with open(TXT_PATH, "r") as file:
    for num, line in enumerate(file):
        if 'jpg' in line:
            line = line.split(' ')[-1].rstrip()
            filename_list.append(line)
        else:
            line = line.split(' ', maxsplit=3)[-1].rstrip()
            results_list.append(line)
            
# results_list = results_list[0:-1]

In [5]:
filename_list

['148.jpg',
 '177.jpg',
 '083.jpg',
 '164.jpg',
 '076.jpg',
 '146.jpg',
 '079.jpg',
 '065.jpg',
 '125.jpg',
 '165.jpg',
 '113.jpg',
 '180.jpg',
 '149.jpg',
 '081.jpg']

In [6]:
len(filename_list)

14

In [7]:
results_list

["{'id': 0, 'barcode': '2860602715096', 'doc_type': 'Товарнаян накладная', 'doc_number': '8017423142', 'error_list': []}",
 "{'id': 1, 'barcode': '2860617614391', 'doc_type': None, 'doc_number': '6017644991', 'error_list': []}",
 "{'id': 2, 'barcode': '2810096516426', 'doc_type': 'Товарная накладная', 'doc_number': '6010594120', 'error_list': []}",
 "{'id': 3, 'barcode': '2860611204697', 'doc_type': 'Товарнаям накладная', 'doc_number': 'B047539166', 'error_list': []}",
 "{'id': 4, 'barcode': '2810096516365', 'doc_type': 'Товарная накладная', 'doc_number': None, 'error_list': []}",
 "{'id': 5, 'barcode': '2860601473409', 'doc_type': None, 'doc_number': '8817415146', 'error_list': []}",
 "{'id': 6, 'barcode': '2810097409826', 'doc_type': 'Товарная накладная', 'doc_number': None, 'error_list': []}",
 "{'id': 7, 'barcode': '2810097409826', 'doc_type': 'Товарная накладная', 'doc_number': None, 'error_list': []}",
 "{'id': 8, 'barcode': '2860582098769', 'doc_type': 'Товарнаян гнакладная', 'd

In [8]:
len(results_list)

14

In [9]:
df = pd.DataFrame()

for num, _ in enumerate(results_list):
    q = eval(results_list[num].replace('\\', ''))
    if q['error_list'] == []:
        q['error_list'] = 0
    else:
        q['error_list'] = 1
    
    res_df = pd.DataFrame(q, index=[0])
    df = pd.concat([df, res_df])

In [10]:
df = df.reset_index(drop=True)

In [11]:
df['filename'] = filename_list

In [12]:
df = df[['id', 'filename', 'barcode', 'doc_type', 'doc_number', 'error_list']]

In [13]:
df.head()

,id,filename,barcode,doc_type,doc_number,error_list
0,0,148.jpg,2860602715096,Товарнаян накладная,8017423142,0
1,1,177.jpg,2860617614391,None,6017644991,0
2,2,083.jpg,2810096516426,Товарная накладная,6010594120,0
3,3,164.jpg,2860611204697,Товарнаям накладная,B047539166,0
4,4,076.jpg,2810096516365,Товарная накладная,None,0


## Визуализация

## Анализ данных

### Необходимые функции

In [14]:
def not_numeric_in_colunm(col):
    not_numeric = {}
    for i, value in enumerate(col):
        if col[i].isnumeric() == False:
            not_numeric[i] = value
    return not_numeric

In [15]:
def len_of_numbers(col):
    length_dict = {}
    for i, value in enumerate(col):
        if value.isnumeric():
            if len(value) not in length_dict.keys():
                length_dict[len(value)] = 1
            else:
                length_dict[len(value)] += 1
    return length_dict

### Error list = 1

Если в столбце **error_list** 0, значит изображение прошло проверку детектором блюра и предварительным распознавателем текста (определение ортогонального положения). 

Если в этом столбце 1, значит имеет место одна или две ошибки. следовательно изображение не будет распознано. Необходимо оценить качество работы детектора блюра и ортогонального оперделителя. 

In [16]:
df['error_list'].value_counts()

0    14
Name: error_list, dtype: int64

Итого не будет распознано **9** изображений. Оценим их.

In [17]:
error_df = df[df['error_list'] == 1]
error_df

,id,filename,barcode,doc_type,doc_number,error_list


**Вывод**

Можно сказать, что деткетор блюра позволил отсеять размытые изображения, их загрузка в SAP нежелательная, поскольку большая часть информации в них потеряна.

### Error list = 0

In [18]:
good_df = df[df['error_list'] == 0]
good_df.head(10)

,id,filename,barcode,doc_type,doc_number,error_list
0,0,148.jpg,2860602715096,Товарнаян накладная,8017423142,0
1,1,177.jpg,2860617614391,None,6017644991,0
2,2,083.jpg,2810096516426,Товарная накладная,6010594120,0
3,3,164.jpg,2860611204697,Товарнаям накладная,B047539166,0
4,4,076.jpg,2810096516365,Товарная накладная,None,0
5,5,146.jpg,2860601473409,None,8817415146,0
6,6,079.jpg,2810097409826,Товарная накладная,None,0
7,7,065.jpg,2810097409826,Товарная накладная,None,0
8,8,125.jpg,2860582098769,Товарнаян гнакладная,6Q17073063,0
9,9,165.jpg,2860611233796,оварнаяя накледная,5041753049,0


In [19]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14 non-null     int64 
 1   filename    14 non-null     object
 2   barcode     14 non-null     object
 3   doc_type    12 non-null     object
 4   doc_number  7 non-null      object
 5   error_list  14 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 784.0+ bytes


In [20]:
good_df  = good_df.fillna('None')

In [21]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          14 non-null     int64 
 1   filename    14 non-null     object
 2   barcode     14 non-null     object
 3   doc_type    14 non-null     object
 4   doc_number  14 non-null     object
 5   error_list  14 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 784.0+ bytes


In [22]:
good_df = good_df.reset_index(drop=True)

**Вывод**

Далее будем работать с этим датасетом. Отфильтрованные изображением считаем истинно-положительными.

### Barcode

In [23]:
bad_barcode_df = good_df[good_df['barcode'] == 'None'].reset_index(drop=True)
bad_barcode_df

,id,filename,barcode,doc_type,doc_number,error_list


In [24]:
barcode_not_numeric = not_numeric_in_colunm(good_df['barcode'])
barcode_not_numeric

{}

In [25]:
length_of_barcode_numbers = len_of_numbers(good_df['barcode'])
length_of_barcode_numbers

{13: 14}

In [26]:
length_of_barcode_numbers[13] / len(good_df)

1.0

In [27]:
bad_barcode_list = list(bad_barcode_df['filename'])
bad_barcode_list

[]

#### Вывод 
В столбце со штрих-кодами 0 пропущенных значений **None**. Остальные значения соответствуют десятизначным штрих-кодам. Итого можно считать распознанными **100%** штрих-кодов. Из исходного датасета необходимо исключить 10 изображений, которые содержали хотябы одну ошибку в списке error_list.

### Doc type

In [28]:
good_df['doc_type'].unique()

array(['Товарнаян накладная', 'None', 'Товарная накладная',
       'Товарнаям накладная', 'Товарнаян гнакладная',
       'оварнаяя накледная', 'товарнаян накладная', 'Товарнаян нактадная'],
      dtype=object)

In [29]:
good_df['doc_type'].value_counts()

Товарная накладная      5
None                    2
Товарнаян гнакладная    2
Товарнаян накладная     1
Товарнаям накладная     1
оварнаяя накледная      1
товарнаян накладная     1
Товарнаян нактадная     1
Name: doc_type, dtype: int64

In [30]:
1 - len(good_df[good_df['doc_type'] == 'None']) / len(df)

0.8571428571428572

In [31]:
bad_doc_type_df = good_df[good_df['doc_type'] == 'None']
bad_doc_type_df.head()

,id,filename,barcode,doc_type,doc_number,error_list
1,1,177.jpg,2860617614391,None,6017644991,0
5,5,146.jpg,2860601473409,None,8817415146,0


In [32]:
bad_doc_type_list = list(bad_doc_type_df['filename'])
sorted(bad_doc_type_list)

['146.jpg', '177.jpg']

#### Вывод 
В столбце с типами документов 3 пропущенных значения **None**. Также в этом столбце есть почти распознанные данные, например **Товарная накладнея**, **товарная накладнаяi** и другие. В почти распознанных данных допущено по 1 ошибке, так что, можно считать их проходящими по порогу распознавания. Итого качественно распознано 190 типов документов из 193 примеров, или всего **98,5%**.

### Doc number

In [33]:
bad_doc_number = good_df[good_df['doc_number'] == 'None']
bad_doc_number.head(15)

,id,filename,barcode,doc_type,doc_number,error_list
4,4,076.jpg,2810096516365,Товарная накладная,None,0
6,6,079.jpg,2810097409826,Товарная накладная,None,0
7,7,065.jpg,2810097409826,Товарная накладная,None,0
10,10,113.jpg,2860597420210,Товарнаян гнакладная,None,0
11,11,180.jpg,2860618473492,товарнаян накладная,None,0
12,12,149.jpg,2860609765162,Товарнаян нактадная,None,0
13,13,081.jpg,2810096516365,Товарная накладная,None,0


In [34]:
bad_doc_number_df = good_df[good_df['doc_number'].str.isnumeric() == False]
bad_doc_number_df.tail(20)

,id,filename,barcode,doc_type,doc_number,error_list
3,3,164.jpg,2860611204697,Товарнаям накладная,B047539166,0
4,4,076.jpg,2810096516365,Товарная накладная,None,0
6,6,079.jpg,2810097409826,Товарная накладная,None,0
7,7,065.jpg,2810097409826,Товарная накладная,None,0
8,8,125.jpg,2860582098769,Товарнаян гнакладная,6Q17073063,0
10,10,113.jpg,2860597420210,Товарнаян гнакладная,None,0
11,11,180.jpg,2860618473492,товарнаян накладная,None,0
12,12,149.jpg,2860609765162,Товарнаян нактадная,None,0
13,13,081.jpg,2810096516365,Товарная накладная,None,0


In [35]:
len(bad_doc_number_df)

9

In [36]:
bad_doc_number_df['doc_number'].value_counts()

None          7
B047539166    1
6Q17073063    1
Name: doc_number, dtype: int64

In [37]:
doc_number_not_numeric = not_numeric_in_colunm(good_df['doc_number'])
doc_number_not_numeric

{3: 'B047539166',
 4: 'None',
 6: 'None',
 7: 'None',
 8: '6Q17073063',
 10: 'None',
 11: 'None',
 12: 'None',
 13: 'None'}

In [38]:
length_of_doc_numbers = len_of_numbers(good_df['doc_number'])
length_of_doc_numbers

{10: 5}

In [39]:
for i in good_df['doc_number']:
    if len(i) > 10:
        print(i)

In [40]:
length_of_doc_numbers[10]/len(good_df)

0.35714285714285715

In [41]:
bad_doc_number_list = list(bad_doc_number_df['filename'])
sorted(bad_doc_number_list)

['065.jpg',
 '076.jpg',
 '079.jpg',
 '081.jpg',
 '113.jpg',
 '125.jpg',
 '149.jpg',
 '164.jpg',
 '180.jpg']

In [42]:
len(bad_doc_number_list)

9

#### Вывод 
В столбце с номерами документов 124 пропущенных значения. В них фигурируют либо **None** либо нечисловые значения, например **Konmueerue**. Качественно распознано 79 номеров документов из 203 примеров, или всего 38,9%.

## Результат исследования

In [43]:
bad_lists_union = sorted(list(set().union(bad_doc_type_list, bad_doc_number_list)))

In [44]:
len(bad_lists_union)

11

In [45]:
len(good_df) - len(bad_lists_union)

3

In [46]:
1 -len(bad_lists_union) / len(good_df)

0.2142857142857143

## Общий вывод

**Итерация 1** - отсутствует предобработка (первый препроцессинг). Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 2** - добавлен второй препроцессинг. Можно считать качественно распознанными 178 документов из 193, что составляет **92.70%** из тестовых данных.

**Итерация 3** - добавлен третий препроцессинг. Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 4** - добавлен четвертый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 5** - добавлен пятый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 6** - добавлен новый препроцессинг. 

`return cv2.GaussianBlur(img, (1, 1), 0)`

Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

In [47]:
bad_lists_union

['065.jpg',
 '076.jpg',
 '079.jpg',
 '081.jpg',
 '113.jpg',
 '125.jpg',
 '146.jpg',
 '149.jpg',
 '164.jpg',
 '177.jpg',
 '180.jpg']

In [4]:
bad_doc_type = ['146.jpg', '177.jpg']

In [9]:
for i in bad_doc_type:
    image = cv2.imread(DATA_PATH + i)
    kernel = np.ones((1, 1), 'uint8')

    image = cv2.dilate(image, kernel, iterations=2)
    image = cv2.GaussianBlur(image, (5, 5), 4)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, 190, 255, cv2.THRESH_BINARY)  # 130
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow(f'Image {i}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()